# NNUE training

## Model architecture

Current model architecture is a sparse, binary array of length 768. Each element of the array represents a possible combination of piece type (6), piece_color (2) and position (64) (6*2*64 = 768).  

The fully connected feedfoward network has 2 hidden layers: 768 -> 8, 8 -> 8 and 8 -> 1.

The output is a single scalar.

Currently used training data: https://www.kaggle.com/competitions/train-your-own-stockfish-nnue/data

In [ ]:
## Input data
### Load input data

In [2]:
import pandas as pd

input_path = '/home/yvlaere/projects/yvl-chess/NNUE_training/training_data/train.csv'

training_df = pd.read_csv(input_path)

print(training_df.head())

                                                 FEN  Evaluation
0  r1b2rk1/ppp2pbp/3q1np1/n3p1B1/2B5/1Q3N2/PP1N1P...        -135
1  8/1pp2p2/6k1/4P2p/p1PR1K1P/2r2P2/6P1/8 w - - 0 33         -57
2  r2qk1nr/1b3pbp/n3p1p1/1pp1P3/p2PN3/2P2N2/PPB3P...         541
3  2b2rk1/5pp1/p2q1n1p/P2pn3/3N4/3BP1B1/2Q2PPP/Rr...         163
4  r2qkb1r/ppp2ppb/2n1p3/3n2PQ/3Pp3/2P4P/PP6/RNB1...        -332


### Turn FEN into input layer

In [ ]:
import numpy as np

piece_dict = {'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5, 'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11}

def FEN_to_input(fen):
    """
    Convert a FEN string to an NNUE input vector.
    """
    # Split the FEN string into its components
    sub_FEN = fen.split(' ')
    board = sub_FEN[0]
    ranks = board.split('/')

    # Convert the board to a 1D boolean array
    # in the chess engine, position 0 corresponds to a1, so the ranks in the FEN string will need to be reversed
    input_layer = np.zeros(768, dtype = np.bool)
    position = 0
    for rank in ranks[::-1]:
        for char in rank:
            if char.isdigit():
                position += int(char)
            else:
                input_layer[position*piece_dict[char]] = True

    return input_layer